Importando as pacotes

In [20]:
import pandas as pd

Lendo os arquivos

In [21]:
df_orders = pd.read_csv('dataset/olist_orders_dataset.csv.zip', compression='zip')
df_reviews = pd.read_csv('dataset/olist_order_reviews_dataset.csv.zip', compression='zip')
df_customers = pd.read_csv('dataset/olist_customers_dataset.csv.zip', compression='zip')
df_geolocation = pd.read_csv('dataset/olist_geolocation_dataset.csv.zip', compression='zip')
df_items = pd.read_csv('dataset/olist_order_items_dataset.csv.zip', compression='zip')
df_payments = pd.read_csv('dataset/olist_order_payments_dataset.csv.zip', compression='zip')
df_products = pd.read_csv('dataset/olist_products_dataset.csv.zip', compression='zip')

Verificando os tipos

In [22]:
df_orders.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [23]:
df_reviews.dtypes

review_id                  object
order_id                   object
review_score                int64
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object

Tratando as datas

In [24]:
df_customers.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [25]:
df_geolocation.dtypes

geolocation_zip_code_prefix      int64
geolocation_lat                float64
geolocation_lng                float64
geolocation_city                object
geolocation_state               object
dtype: object

In [26]:
df_items.dtypes

order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

In [27]:
df_payments.dtypes

order_id                 object
payment_sequential        int64
payment_type             object
payment_installments      int64
payment_value           float64
dtype: object

In [28]:
df_products.dtypes

product_id                     object
product_category_name          object
product_name_lenght           float64
product_description_lenght    float64
product_photos_qty            float64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object

In [29]:
#Base de Orders
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'], format = '%Y/%m/%d %H:%M:%S.%f')
df_orders['order_approved_at'] = pd.to_datetime(df_orders['order_approved_at'], format = '%Y/%m/%d %H:%M:%S.%f')
df_orders['order_delivered_carrier_date'] = pd.to_datetime(df_orders['order_delivered_carrier_date'], format = '%Y/%m/%d %H:%M:%S.%f')
df_orders['order_delivered_customer_date'] = pd.to_datetime(df_orders['order_delivered_customer_date'], format = '%Y/%m/%d %H:%M:%S.%f')
df_orders["order_estimated_delivery_date"] = pd.to_datetime(df_orders["order_estimated_delivery_date"], format = '%Y/%m/%d').dt.date

#Base de Reviews
df_reviews["review_creation_date"] = pd.to_datetime(df_reviews["review_creation_date"], format = '%Y/%m/%d').dt.date
df_reviews['review_answer_timestamp'] = pd.to_datetime(df_reviews['review_answer_timestamp'], format = '%Y/%m/%d %H:%M:%S.%f')

In [30]:
df = pd.merge(df_orders, df_reviews, how = 'left', on = 'order_id')

In [31]:
df_orders = pd.merge(df_orders, df_reviews, how = 'left', on = 'order_id')
df_orders = pd.merge(df_orders, df_items, how = 'left', on = 'order_id')
df_orders = pd.merge(df_orders, df_products, how = 'left', on = 'product_id')

In [32]:
df['order_id'].value_counts()

8e17072ec97ce29f0e1f111e598b0c85    3
03c939fd7fd3b38f8485a0f95798f1f6    3
c88b1d1b157a9999ce368f218a407141    3
df56136b8031ecd28e200bb18e6ddb2e    3
95442deb81a5d91c97c0df96b431634a    2
                                   ..
94a1020970476388adf12c46628499be    1
b266c12d9bececa3942c1bef84f13716    1
e82e0697786dcef38c215ec9d2dc308a    1
e940e33cefca8e48b759474760c2fb41    1
66dea50a8b16d9b4dee7af250b4be1a5    1
Name: order_id, Length: 99441, dtype: int64

In [33]:
from plotly import tools
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import colorlover as cl
# Others
import warnings
# Product value by date
df_orders['datetime'] =  pd.to_datetime(df_orders['order_purchase_timestamp'])
value_date = df_orders.groupby([df_orders['datetime'].dt.date])['price'].sum()
freight_date = df_orders.groupby([df_orders['datetime'].dt.date])['freight_value'].sum()
# Plot timeseries
trace0 = go.Scatter(x=value_date.index.astype(str), y=value_date.values, opacity = 0.8, name='Product value')
trace1 = go.Scatter(x=freight_date.index.astype(str), y=freight_date.values, opacity = 0.8, name='Freight value')
layout = dict(
    title= "Product and freight value by date",
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1m', step='month', stepmode='backward'),
                dict(count=6, label='6m', step='month', stepmode='backward'),
                dict(count=12, label='12m', step='month', stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible = True),
        type='date'
    )
)
fig = dict(data= [trace0, trace1], layout=layout)
iplot(fig)

# Sales for month
value_month = df_orders[['datetime', 'price']].copy()
value_month.set_index('datetime', inplace=True)
value_month = value_month.groupby(pd.Grouper(freq="M"))['price'].sum()
trace = go.Bar(x= value_month.index, y= value_month.values)
layout = go.Layout(title='Sales per month (product value)', height=420, width=800)
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)
